In [7]:
from external.tab_pfn_gen.src.tabpfgen.tabpfgen import TabPFGen
import torch
from performance_metrics.measure_model_performance import measure_model_performance, measure_distance_to_nearest_neighbour
from performance_metrics.measure_model_performance import  measure_regresion_model_performance, measure_k_anonimity


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cpu


# Test quality

In [ ]:
generator = TabPFGen(n_sgld_steps=200, device=device)
print(measure_model_performance(generator.generate_classification))

In [ ]:
generator = TabPFGen(n_sgld_steps=200, device=device)
print(measure_regresion_model_performance(generator.generate_regression))

In [ ]:
generator = TabPFGen(n_sgld_steps=200, device=device)
print(measure_distance_to_nearest_neighbour(generator.generate_regression))

In [ ]:
generator = TabPFGen(n_sgld_steps=200, device=device)
print(measure_k_anonimity(generator.generate_regression))